In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics

In [2]:
reib_daten = pd.read_csv('data_2019-26-11_REIB.csv')
achsleistung_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/Achsleistung-2019-11-20T10-27-03.csv', delimiter=';')
messprotokoll_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/MEAS_PROTOCOL_CSV.CSV', delimiter=';')
messergebnisse_daten = pd.read_csv('../Messergebnisse Reiben/Messergebnisse.csv', skiprows=7, delimiter=';')
#messergebnisseme_daten = pd.read_csv('../Messergebnisse Reiben/MessergebnisseME01.csv', delimiter=';')
achsleistung_columns = ['Timestamp', 'X1', 'Y1', 'Z1', 'SPI1']
messergebnisse_columns = ['Messwert Innenmessschraube', 'Messwert Messtaster', 'Klassifizierung nach Messtaster', 'Klassifizierung nach Messtaster', 'Klassifizierung nach Grenzlehrdorn']
achsleistung_daten = achsleistung_daten[achsleistung_columns]
messergebnisse_daten = messergebnisse_daten[messergebnisse_columns]


achsleistung_daten = achsleistung_daten.dropna()
messprotokoll_daten = messprotokoll_daten.dropna()
messergebnisse_daten = messergebnisse_daten.dropna()

#achsleistung_daten
#reib_daten
#messprotokoll_daten
#messergebnisse_daten
#messprotokoll_daten

In [ ]:
# def createDataFrame(number, data):
#     number = pd.DataFrame(data)
#     return number


# for index in range(len(messergebnisse_daten)):
#     for key, value in messergebnisse_daten.items():
#         temp = []
#         temp.append(value)
#         if (index % 81) == 0:
#             number = index % 81
#             createDataFrame(number, )
#             new_df.append(temp)
#             #print(new_df.head())

In [5]:
# messergebnisse_daten.head(), messprotokoll_daten.head()
messprotokoll_condensed_median = pd.DataFrame()
for key, value in messprotokoll_daten.iteritems():
    measured = []
    diameter = []
    #print(key)
    if key == 'Measured':
        for i in range(3):
            measured.append(value[i])
            print(value)
    elif key == 'Difference':
        for key in range(3):
            measured.append(messprotokoll_daten[key].value)
    
    else:
        print('Das kann kein Messprotokoll sein!')
    
    #median = statistics.median(temp)
    #messprotokoll_condensed_median.append()



merge_csv_erg_prot = messergebnisse_daten[:81].merge(messprotokoll_daten, right_index=True , left_index=True)

#merge_csv_erg_prot

Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
Das kann kein Messprotokoll sein!
